In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from PIL import Image
from numpy.linalg import inv
import cv2
import time

import sys
sys.path.insert(1, '..')
from DON_Picking.Configuration import config,SINGLE_OBJECT_WITHIN_SCENE

from DON_Training.DataGenerator_O2O import DataGenerator,loadAllModels,convertToHeatmap
from ITRIP.objects_new import graspnet_train
import matplotlib.pyplot as plt

scaleCmToPx 0.09598826886969584


In [2]:
path = "../../dataDON/test_seen/scene_0105"
listObj = np.loadtxt(path+"/object_id_list.txt").astype(np.uint8)
print (listObj.shape)
print (listObj)

dataGenerator = DataGenerator()
models, colors = loadAllModels(path="../../dataDON/",loadFile=True)

(9,)
[14 17 18  7 21 27 66 63 60]
Loaded 87 models in 0 seconds


In [3]:
start_time = time.time()

end_time = time.time()
print ("Loaded 87 models in ",end_time-start_time," seconds")

Loaded 87 models in  2.8371810913085938e-05  seconds


In [4]:
dataGenerator.setOriginalModelObjects(models, colors)

In [5]:
poseCamera, imageColor, depth, mask, indexMap, segmentMap,uvSegment, arrObjectPose = dataGenerator.getImageData_Real(path+"/realsense",5)

FileNotFoundError: [Errno 2] No such file or directory: '../../dataDON/test_seen/scene_0105/realsense/000005_rgb.jpg'

In [ ]:
print (imageColor.shape)

imageColor = imageColor[:,380:1100,:]

imageColor = cv2.cvtColor(imageColor, cv2.COLOR_BGR2RGB)
newsize = (256,256)
imageColorX = Image.fromarray(imageColor)
imageColorX = imageColorX.resize(newsize)

imageColor = np.array(imageColorX)
print (imageColor.shape)

In [ ]:
from xml.dom import minidom
from transforms3d.quaternions import quat2mat
xmldoc = minidom.parse(path+"/realsense/annotations/0005.xml")
itemlist = xmldoc.getElementsByTagName('obj')
print(len(itemlist))

arrObjectPose = np.zeros(((len(itemlist)+1)*4,4))
i = 1
for item in itemlist:
    #print (item.getElementsByTagName("obj_id").item(0).firstChild.nodeValue)
    #print (item.getElementsByTagName("pos_in_world").item(0).firstChild.nodeValue)
    strQuat =  item.getElementsByTagName("ori_in_world").item(0).firstChild.nodeValue
    strPos = item.getElementsByTagName("pos_in_world").item(0).firstChild.nodeValue
    #print (strQuat)
    quat = np.fromstring(strQuat,sep=' ')
    T =   np.fromstring(strPos,sep=' ')
    
    R =  quat2mat(tuple(quat))
    
    objectPose = np.zeros((4,4))
    objectPose[:3,:3] = R
    objectPose[3,3] = 1
    objectPose[:3,-1] = T
    #print (objectPose)
    arrObjectPose[i*4:(i+1)*4] = objectPose
    i+=1

    
def adjust_color_lightness(r, g, b, factor):
    h, l, s = rgb_to_hls(r / 255.0, g / 255.0, b / 255.0)
    l = max(min(l * factor, 1.0), 0.0)
    r, g, b = hls_to_rgb(h, l, s)
    return int(r * 255), int(g * 255), int(b * 255)


def lighten_color(r, g, b, factor=0.1):
    return adjust_color_lightness(r, g, b, 1 + factor)
from PIL import Image, ImageDraw, ImageFont

from colorsys import rgb_to_hls, hls_to_rgb
img = np.zeros((imageColor.shape))
img = Image.fromarray(np.uint8(img))
renderMap = np.zeros((imageColor.shape[0], imageColor.shape[1]))-10

draw = ImageDraw.Draw(imageColorX)

print (renderMap[50,50])
for obj_th in range (5):
    print (obj_th)
    vertex = models[listObj[obj_th]]
    colorCoord = colors[listObj[obj_th]]
    objPose = arrObjectPose[(obj_th + 1) * 4:(obj_th + 2) * 4]
    #print(objPose)

    intrinsicCamera = np.load(path+"/realsense/camK.npy")

    projectedImage = np.matmul((intrinsicCamera), np.matmul((objPose[:3]), (vertex.T)))
    scale = 1 / projectedImage[2, :]
    projectedImage *= scale

    projectedImage[2, :] = scale
    projectedImage[0, :] -= 380
    
    #projectedImage[:2, :] += 1280//2

    uvCoord = projectedImage.T  / 2.8125
    
    nVertext = uvCoord.shape[0]
    
    step = 3
    
    #print (np.min(vertex[:,2]))

    for i in range(nVertext):
        u = (int)(uvCoord[i, 0])
        v = (int)(uvCoord[i, 1])
        #print (u,v)
        #print (vertex[i,2])  
        if ( 0< u < config["W"] and  0 <  v < config["W"] ):
            if (vertex[i,2] > renderMap[v, u]):
                renderMap[v:v + step, u:u + step] = uvCoord[i, 2]
                r, g, b = (tuple(colorCoord[i]))
                r,g,b = lighten_color(r, g, b, 0.1)

                draw.rectangle((u, v, u + step, v + step), fill=(r,g,b))
#print (uvCoord)


In [ ]:
display(img)
display(imageColorX)
print (segmentMap.shape)
segmentMap = segmentMap[:,380:1100]
print (segmentMap.shape)
segmentMapX = convertToHeatmap(segmentMap)
segmentMapX = segmentMapX.resize((256,256))
mask = np.clip(segmentMap,0,1)
print (depth.shape)
maskX = convertToHeatmap(mask)
display(segmentMapX)
display(maskX)